In [1]:
import pandas as pd

df = pd.read_csv('drug200.csv')
display(df)

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [2]:
display(set(df.loc[:, 'BP']))
display(set(df.loc[:, 'Cholesterol']))
display(set(df.loc[:, 'Drug']))

{'HIGH', 'LOW', 'NORMAL'}

{'HIGH', 'NORMAL'}

{'DrugY', 'drugA', 'drugB', 'drugC', 'drugX'}

In [3]:
df = df.replace(['LOW', 'NORMAL', 'HIGH'], [0, 1, 2])
df = df.replace(['M', 'F'], [1, 0])
df = df.replace(['DrugY', 'drugA', 'drugB', 'drugC', 'drugX'], [0, 1, 2, 3, 4])

display(df)

C:\Users\da02s\AppData\Local\Temp\ipykernel_43404\1691503345.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['LOW', 'NORMAL', 'HIGH'], [0, 1, 2])
C:\Users\da02s\AppData\Local\Temp\ipykernel_43404\1691503345.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['M', 'F'], [1, 0])
C:\Users\da02s\AppData\Local\Temp\ipykernel_43404\1691503345.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `resu

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,2,2,25.355,0
1,47,1,0,2,13.093,3
2,47,1,0,2,10.114,3
3,28,0,1,2,7.798,4
4,61,0,0,2,18.043,0
...,...,...,...,...,...,...
195,56,0,0,2,11.567,3
196,16,1,0,2,12.006,3
197,52,1,1,2,9.894,4
198,23,1,1,1,14.020,4


In [4]:
import statsmodels.formula.api as sm
model = sm.ols('Drug ~ Age + Sex + Cholesterol + Na_to_K', df, subset=None, drop_cols=None)
model_res = model.fit()
print(model_res.summary())

                            OLS Regression Results                            
Dep. Variable:                   Drug   R-squared:                       0.483
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     45.46
Date:                Wed, 14 May 2025   Prob (F-statistic):           6.08e-27
Time:                        11:23:04   Log-Likelihood:                -325.44
No. Observations:                 200   AIC:                             660.9
Df Residuals:                     195   BIC:                             677.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       4.6325      0.426     10.868      

Оценил предикторы, используя p-value (t-критерий Стьюдента здесь не подходит, тк данные не имеют нормального распределения, об этом говорит параметр Omnibus, а еще я для каждого QQ plot строил)  
Если < 0.05, то гипотеза о значимости коэффициента принимается, значит нам подходит только Na_to_K

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(df.loc[:, ['Age', 'Sex', 'BP', 'Na_to_K', 'Cholesterol']], df.loc[:, 'Drug'])

In [7]:
from sklearn import tree
import graphviz

clf = tree.DecisionTreeClassifier()
clf_res = clf.fit(X_train, Y_train)

dot_data = tree.export_graphviz(clf_res, out_file=None)
graph = graphviz.Source(dot_data) 

graph.format = 'PDF'

graph.render('lab_tree') 

'lab_tree.pdf'

In [9]:
from sklearn.metrics import r2_score, mean_squared_error, classification_report

true = Y_test
predict = clf_res.predict(X_test)

# display(df)

errors = {'MSE': [mean_squared_error(true, predict)], 'R^2': [r2_score(true, predict)]}
display(pd.DataFrame(data=errors))
print(classification_report(true, predict))

,MSE,R^2
0,0.32,0.9002


              precision    recall  f1-score   support

           0       0.96      1.00      0.98        25
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         1
           4       1.00      0.94      0.97        16

    accuracy                           0.98        50
   macro avg       0.99      0.99      0.99        50
weighted avg       0.98      0.98      0.98        50



In [10]:
from sklearn.feature_selection import chi2

p_value, chi2_values = chi2(X_train, Y_train)
display(pd.DataFrame(data={'P-values': p_value, 'chi2': chi2_values}))

,P-values,chi2
0,132.172019,1.336111e-27
1,0.212955,9.947181e-01
2,46.781460,1.693401e-09
3,323.574376,8.878453e-69
4,3.675477,4.517027e-01
